# Change Detection (Segmentation) using TAO Visual ChangeNet-Segmentation Network

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080">

## What is a Visual ChangeNet Network?

Visual ChangeNet is a state of the art transformer-based Change Detection model. Similar to the optical inspection notebook, Visual ChangeNet is also based on Siamese Network, which is a class of neural network architectures containing two or more identical subnetworks. The training algorithm works by updating the parameters across all the sub-networks in tandem. In TAO, Visual ChangeNet supports two images as input where the end goal is to either classify or segment the change between the "golden or reference" image and the "test" image. TAO supports the [FAN](https://arxiv.org/abs/2204.12451) backbone network for both Visual ChangeNet architectures. For more details about training FAN backbones, please refer to the Classification pytorch notebook.

TAO Toolkit versions 5.3 and later support some of the foundational models for change detection (classification and segmentation). NV-DINOv2 can now be used as the backbone for the Visual ChangeNet-Classification and Segmentation models.

To mitigate the inferiror performance of a standard vision transformer (ViT) on dense prediction tasks, TAO supports the ViT-Adapter architecture. This allows a powerful ViT that has learned rich semantic representations from a large corpus of data to achieve comparable performance to vision-specific transformers on dense preidiction tasks.

In TAO, two different types of Change Detection networks are supported: Visual ChangeNet-Segmentation and Visual ChangeNet-Classification intended for Segmentation and Classification of change between the two input images, respectively. Visual ChangeNet-Segmentation is specifically intended for segmentation change detection. In this notebook, we leverage the Visual ChangeNet-Segmentation model to demonstrate change detection by segmenting the changed pixels between the two input images. 

### Sample inference from the Visual ChangeNet-Segmentation model

Here is a sample inference output for the Visual ChangeNet-Segmentation model. The model takes in two input images (test and golden reference) and outputs a segmentation change map for building change detection on LEVIR-CD dataset.

<img align="center" title="no defects" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/purpose_built_models/visual_changenet/sample_image.jpg?raw=true" width="800" >


## Learning Objectives

In this notebook, you will learn how to leverage the simplicity and convenience of TAO to:

* Train a Visual ChangeNet-Segmentation Model for Semantic change detection on remote sensing imagery.
* Evaluate the trained model.
* Run Inference on the trained model.
* Export the trained model to a .onnx file (encrypted ONNX model) for deployment to DeepStream or TensorRT.

At the end of this notebook, you will have generated a trained and optimized `visual_changenet` model, 
which you may deploy with this [end-to-end sample](https://github.com/NVIDIA-AI-IOT/tao-toolkit-triton-apps) with Triton.


## Table of Contents

This notebook shows an example usecase of Siamese Network using Train Adapt Optimize (TAO) Toolkit.

0. [Set up env variables and map drives](#head-0)
1. [Installing the TAO launcher](#head-1)
2. [Prepare dataset](#head-2)
3. [Download the pretrained model from NGC](#head-3)
4. [Provide training specification](#head-4)
5. [Run TAO training](#head-5)
6. [Evaluate trained models](#head-6)
7. [Inferences](#head-7)
8. [Deploy](#head-8)


## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>

The TAO launcher uses docker containers under the hood, and **for our data and results directory to be visible to the docker, they need to be mapped**. The launcher can be configured using the config file `~/.tao_mounts.json`. Apart from the mounts, you can also configure additional options like the Environment Variables and amount of Shared Memory available to the TAO launcher. <br>

`IMPORTANT NOTE:` The code below creates a sample `~/.tao_mounts.json`  file. Here, we can map directories in which we save the data, specs, results and cache. You should configure it for your specific case so these directories are correctly visible to the docker container.


In [ ]:
import os

# Please define this local project directory that needs to be mapped to the TAO docker session.
%env LOCAL_PROJECT_DIR=FIXME

os.environ["HOST_DATA_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "data", "changenet")
os.environ["HOST_RESULTS_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "changenet", "results")
os.environ["HOST_MODEL_DIR"] = os.path.join(os.getenv("LOCAL_PROJECT_DIR", os.getcwd()), "changenet", "model")

# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=/path/to/local/tao-experiments/changenet
# The sample spec files are present in the same path as the downloaded samples.
os.environ["HOST_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)

In [ ]:
! mkdir -p $HOST_DATA_DIR
! mkdir -p $HOST_SPECS_DIR
! mkdir -p $HOST_RESULTS_DIR
! mkdir -p $HOST_MODEL_DIR

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")
tlt_configs = {
   "Mounts":[
       # Mapping the data directory
       {
           "source": os.environ["LOCAL_PROJECT_DIR"],
           "destination": "/workspace/tao-experiments"
       },
       {
           "source": os.environ["HOST_DATA_DIR"],
           "destination": "/data"
       },
       {
           "source": os.environ["HOST_MODEL_DIR"],
           "destination": "/model"
       },
       {
           "source": os.environ["HOST_SPECS_DIR"],
           "destination": "/specs"
       },
       {
           "source": os.environ["HOST_RESULTS_DIR"],
           "destination": "/results"
       }
   ],
   "DockerOptions": {
        "shm_size": "16G",
        "ulimits": {
            "memlock": -1,
            "stack": 67108864
         }
   }
}
# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(tlt_configs, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Installing the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a python package distributed as a python wheel listed in PyPI. You may install the launcher by executing the following cell.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python 3.7.0. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the `virtualenv` and `virtualenvwrapper` packages. Once you have setup virtualenvwrapper, please set the version of python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 7 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO python package, please make sure of the following software requirements:
* python >=3.7.0 < 3.8.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 525+

Once you have installed the pre-requisites, please log in to the docker registry nvcr.io by following the command below

```sh
docker login nvcr.io
```

You will be triggered to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Please follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

Please note that TAO Toolkit recommends users to run the TAO launcher in a virtual env with python >=3.7.0. You may follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a python virtual env using the virtualenv and virtualenvwrapper packages.

In [ ]:
# SKIP this step IF you have already installed the TAO launcher.
!pip3 install nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info --verbose

## 2. Prepare a sample dataset <a class="anchor" id="head-2"></a>

TAO Visual ChangeNet-Segmentation uses a custom dataset format. The sections below walk through this format, specifically how the dataset should be structured and the files required.

### Visual ChangeNet-Segmentation data format

Visual ChangeNet-Segmentation expects directories of images and mask files in the dataset root directory. The image directories consist of
golden image directory (pre-change images) and test image directory (post-change image) to compare against the change mask images with pixel level change masks.  


     |--dataset_root:
          |--A
               |--image1.jpg
               |--image2.jpg
          |--B
               |--image1.jpg
               |--image2.jpg
          |--label
               |--image1.jpg
               |--image2.jpg      
          |--list
               |-- train.txt
               |-- val.txt
               |-- test.txt
               |-- predict.txt

Here's a description of the structure:

* The ``dataset_root`` directory contains the following:
    * ``A``: Contains post-change test images
    * ``B``: Contains pre-change golden reference images
    * ``label``: Contains ground truth segmentation change masks
    * ``list``: Contains .txt files for each dataset split, as described in the  section below. 

#### List Files


Visual ChangeNet-Segmentation dataloader expects the ``label`` directory to contain ``.txt`` files for each of the dataset split [train, validation, test, inference]. 
A Visual ChangeNet-Segmentation label file is a simple ``.txt`` file containing all file names for the particular split.


| ``image_names``   | 
| :---------------- |
| ``file_name.png`` |

* ``image_names``: The names of images. Image names should be the same for test images and their corresponding reference and mask images. 

Here is a sample label file corresponding to the sample directory structure as describe in the section above.


| ``image_names``   | 
| :---------------- |
| ``image1.png``    |
| ``image3.png``    |
| ``image2.png``    |

Note: Each test image (inside directory ``A``) must have a reference image (inside directory ``B``)
and a segmentation change map (inside directory ``label``) with the same name for the dataloader to map them correctly.

### 2.1 Download the dataset <a class="anchor" id="head-2-1"></a>

In [ ]:
#Download the data
import os
LEVIR_DOWNLOAD_URL = "https://www.dropbox.com/s/18fb5jo0npu5evm/LEVIR-CD256.zip"
os.environ["URL_DATASET"]=LEVIR_DOWNLOAD_URL
!if [ ! -f $HOST_DATA_DIR/LEVIR-CD256.zip ]; then wget $URL_DATASET -O $HOST_DATA_DIR/LEVIR-CD-256.zip; else echo "image archive already downloaded"; fi 

### 2.2 Verify the downloaded dataset <a class="anchor" id="head-2-2"></a>

In [ ]:
# Check the dataset is present
!mkdir -p $HOST_DATA_DIR
!if [ ! -f $HOST_DATA_DIR/LEVIR-CD-256.zip ]; then echo 'Dataset zip file not found, please download.'; else echo 'Found Dataset zip file.';fi

In [ ]:
# unpack 
!unzip -u $HOST_DATA_DIR/LEVIR-CD-256.zip -d $HOST_DATA_DIR

In [ ]:
import os
import shutil
def verify_dataset(data_dir):

    num_images_A = len(os.listdir(os.path.join(DATA_DIR, "LEVIR-CD256/A")))
    num_images_B = len(os.listdir(os.path.join(DATA_DIR, "LEVIR-CD256/B")))
    num_labels = len(os.listdir(os.path.join(DATA_DIR, "LEVIR-CD256/label")))
    num_train_images = sum(1 for _ in open(os.path.join(DATA_DIR, "LEVIR-CD256/list/train.txt")))
    num_val_images = sum(1 for _ in open(os.path.join(DATA_DIR, "LEVIR-CD256/list/val.txt")))
    num_test_images = sum(1 for _ in open(os.path.join(DATA_DIR, "LEVIR-CD256/list/test.txt")))
    print("Number of images in the train/val/test set. {}".format(num_images_A))
    print("Number of compare images in the train/val/test set. {}".format(num_images_B))
    print("Number of labels in the train/val/test set. {}".format(num_labels))
    print("Number of train instances. {}".format(num_train_images))
    print("Number of val instances. {}".format(num_val_images))
    print("Number of test instances. {}".format(num_test_images))

    # # Use test set for inference
    # shutil.copy(os.path.join(DATA_DIR, "LEVIR-CD-256/list/test.txt"), os.path.join(DATA_DIR, "LEVIR-CD-256/list/predict.txt"))

DATA_DIR = os.environ.get('HOST_DATA_DIR')
# verify downloaded dataset
verify_dataset(data_dir=DATA_DIR)

In [ ]:
# Verify
!ls -l $HOST_DATA_DIR/

# 3. Download the pretrained model from NGC <a class="anchor" id="head-3"></a>

Download the pretrained model from NGC. We will use NGC CLI to get the data and model. For more details, go to https://ngc.nvidia.com and click the SETUP on the navigation bar.

In [ ]:
# Installing NGC CLI on the local machine.
## Download and install
import os
%env CLI=ngccli_cat_linux.zip
!mkdir -p $HOST_RESULTS_DIR/ngccli

# # Remove any previously existing CLI installations
!rm -rf $HOST_RESULTS_DIR/ngccli/*
!wget "https://ngc.nvidia.com/downloads/$CLI" -P $HOST_RESULTS_DIR/ngccli
!unzip -u "$HOST_RESULTS_DIR/ngccli/$CLI" -d $HOST_RESULTS_DIR/ngccli/
!rm $HOST_RESULTS_DIR/ngccli/*.zip
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format(os.getenv("HOST_RESULTS_DIR", ""), os.getenv("PATH", ""))

In [ ]:
!ngc registry model list nvidia/tao/visual_changenet_segmentation_levircd:*

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/pretrained

In [ ]:
!ngc registry model download-version "nvidia/tao/visual_changenet_segmentation_levircd:visual_changenet_levircd_trainable_v1.0" --dest $HOST_RESULTS_DIR/pretrained

In [ ]:
print("Check that model is downloaded into dir.")
!ls -l $HOST_RESULTS_DIR/pretrained/visual_changenet_segmentation_levircd_vvisual_changenet_levircd_trainable_v1.0/

## 4. Provide training specification <a class="anchor" id="head-4"></a>

We provide specification files to configure the training parameters including:

* task: segment
* train:
  * resume_training_checkpoint_path: null
  * pretrained_model_path: /path/to/pretrained/model.pth
  * segment:
    * loss: "ce"
    * weights: [0.5, 0.5, 0.5, 0.8, 1.0]
  * num_epochs: 350
  * num_nodes: 1
  * val_interval: 1
  * checkpoint_interval: 1
  * optim:
    * lr: 0.0002
    * optim: "adamw"
    * policy: "linear" 
    * momentum: 0.9
    * weight_decay: 0.01
    * betas: [0.9, 0.999]
* results_dir: "/results"
* model:
  * backbone:
    * type: "fan_small_12_p4_hybrid"
    * pretrained_backbone_path: /path/to/pretrained/backbone.pth
    * freeze_backbone: False
  * decode_head:
    * feature_strides: [4, 8, 16, 16]
* dataset:
  * segment:
    * dataset: "CNDataset"
    * root_dir: /data/LEVIR-CD-256
    * data_name: "LEVIR-CD"
    * label_transform: "norm"
    * batch_size: 16
    * workers: 2
    * multi_scale_train: True
    * multi_scale_infer: False
    * num_classes: 2
    * img_size: 256
    * image_folder_name: "A"
    * change_image_folder_name: 'B'
    * list_folder_name: 'list'
    * annotation_folder_name: "label"
    * train_split: "train"
    * validation_split: "val"
    * test_split: 'test'
    * predict_split: 'predict'
    * label_suffix: .png
    * augmentation: 
      * random_flip:
        * vflip_probability: 0.5
        * hflip_probability: 0.5
        * enable: True
      * random_rotate:
        * rotate_probability: 0.5
        * angle_list: [90, 180, 270]
        * enable: True
      * random_color:
        * brightness: 0.3
        * contrast: 0.3
        * saturation: 0.3
        * hue: 0.3
        * enable: True
      * with_scale_random_crop:
        * enable: True
      * with_random_crop: True
      * with_random_blur: True

Please refer to the TAO documentation about Visual ChangeNet-Segmentation to get all the parameters that are configurable.

In [ ]:
!cat $HOST_SPECS_DIR/experiment.yaml

## 5. Run TAO training <a class="anchor" id="head-5"></a>
* Provide the sample spec file and the output directory location for models.
* WARNING: Training will take several hours or one day to complete.
* LEVIR-CD per-epoch training time on a single GPU (the hours may vary depending on the data location, network speed, and etc).

<table>
  <tr>
    <th>GPU Type</th>
    <th>Time (minutes)</th>
  </tr>
  <tr>
    <td>1 x A6000 48GB</td>
    <td>12</td>
  </tr>
</table>

* If you're looking to speed through the notebook, you can adjust the `train.num_epochs` parameter for a quicker run. It's crucial to understand that this adjustment is intended for demonstration and educational purposes only and won't produce state-of-the-art (SOTA) results.
* To replicate baseline numbers accurately, the model should be trained for 300 epochs.

In [ ]:
# NOTE: The following paths are set from the perspective of the TAO Docker.
# The data is saved here
%env DATA_DIR = /data
%env MODEL_DIR = /model
%env SPECS_DIR = /specs
%env RESULTS_DIR = /results
%env NUM_EPOCHS=5

### 5.1 Train Visual ChangeNet-Segmentation model

We will train a Visual ChangeNet-Segmentation model from scratch.

In [ ]:
print("Train model")
!tao model visual_changenet train \
                  -e $SPECS_DIR/experiment.yaml \
                    train.pretrained_model_path=$RESULTS_DIR/pretrained/visual_changenet_segmentation_levircd_vvisual_changenet_levircd_trainable_v1.0/changenet_segment_levir_cd.pth
                    # train.num_epochs=$NUM_EPOCHS

In [ ]:
## Training command for multi-gpu training. We can define the number of gpus using the `num_gpus` parameter.
## The following command will trigger multi-gpu training on 2 gpus.
# print("Train model")
# !tao model visual_changenet train \
#                   -e $SPECS_DIR/experiment.yaml \
#                   --gpus 2
#                   train.num_epochs=$NUM_EPOCHS

In [ ]:
print('Model checkpoints:')
!ls -ltrh $HOST_RESULTS_DIR/train

In [ ]:
# You can set NUM_EPOCH to the epoch corresponding to any saved checkpoint
%env NUM_EPOCH=30

In [ ]:
# Get the name of the checkpoint corresponding to your set epoch
tmp=!ls $HOST_RESULTS_DIR/train/*.pth | grep epoch=0$NUM_EPOCH
%env CHECKPOINT={tmp[0]}

# Note: This cell may require sudo access to rename the trained model checkpoint.
print('Rename a trained model: ')
print('---------------------')
!cp $CHECKPOINT $HOST_RESULTS_DIR/train/changenet.pth
!ls -ltrh $HOST_RESULTS_DIR/train/changenet.pth

## 6. Evaluate trained models <a class="anchor" id="head-6"></a>
Evaluate trained model.

For Visual ChangeNet-Segmentation model evaluation, the following metrics are preferred:

* Overall Accuracy.
* mIoU: Intersection over Union (IoU) for all classes.

Note that we provide metrics for Precision, Recall, IoU and F1 score for each individual class as well as averaged over all classes. 

In [ ]:
!tao model visual_changenet evaluate \
                   -e $SPECS_DIR/experiment.yaml \
                    evaluate.checkpoint=$RESULTS_DIR/train/changenet.pth

## 7. Visualize Inference <a class="anchor" id="head-7"></a>
In this section, we run the Visual ChangeNet-Segmentation inference tool to generate inferences with the trained models and save the results under `$RESULTS_DIR`. We also visualise a few sample outputs along with their corresponding input images. 

In [ ]:
!tao model visual_changenet inference \
                   -e $SPECS_DIR/experiment.yaml \
                    inference.checkpoint=$RESULTS_DIR/train/changenet.pth

In [ ]:
# Simple grid visualizer
!pip3 install matplotlib==3.3.3
import matplotlib.pyplot as plt
import os
from math import ceil
valid_image_ext = ['.jpg']

def visualize_images(output_path, num_cols=4, num_images=10):
    num_rows = int(ceil(float(num_images) / float(num_cols)))
    f, axarr = plt.subplots(num_rows, num_cols, figsize=[80,30])
    f.tight_layout()
    a = [os.path.join(output_path, image) for image in os.listdir(output_path) 
         if os.path.splitext(image)[1].lower() in valid_image_ext]
    for idx, img_path in enumerate(a[:num_images]):
        col_id = idx % num_cols
        row_id = idx // num_cols
        img = plt.imread(img_path)
        axarr[row_id, col_id].imshow(img) 

In [ ]:
# Visualizing the sample images.
IMAGE_DIR = os.path.join(os.environ['HOST_RESULTS_DIR'], "inference")
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(IMAGE_DIR, num_cols=COLS, num_images=IMAGES)

## 8. Deploy <a class="anchor" id="head-8"></a>
Export the model to encrypted ONNX model.

In [ ]:
!mkdir -p $HOST_RESULTS_DIR/export

In [ ]:
!tao model visual_changenet export \
                    -e $SPECS_DIR/experiment.yaml \
                        export.checkpoint=$RESULTS_DIR/train/changenet.pth \
                        export.onnx_file=$RESULTS_DIR/export/changenet.onnx

In [ ]:
# # Uncomment this cell to export an onnx file with dynamic batching enabled for
# # integration with trtexec and deepstream.
# # Export the model with export.batch_size=-1 for dynamic batching.
# ! tao model visual_changenet export \
#                     -e $SPECS_DIR/experiment.yaml \
#                     export.checkpoint=$RESULTS_DIR/train/changenet.pth \
#                     export.batch_size=-1 \
#                     export.results_dir=$RESULTS_DIR/export_dynamic \
#                     export.onnx_file=$RESULTS_DIR/export_dynamic/changenet.onnx
                            

# # Profiling the exported model via trtexec.
# ! tao deploy trtexec --onnx=$RESULTS_DIR/export_dynamic/changenet.onnx \
#                      --minShapes=input0:1x3x512x128,input1:1x3x256x256 \
#                      --optShapes=input0:8x3x512x128,input1:8x3x256x256 \
#                      --maxShapes=input0:16x3x512x128,input1:16x3x256x256 \
#                      --fp16 \
#                      --saveEngine=$RESULTS_DIR/export_dynamic/changenet-fp16.engine

In [ ]:
print('Exported model:')
print('------------')
!ls -lth $HOST_RESULTS_DIR/export

In [ ]:
!tao deploy visual_changenet gen_trt_engine \
            -e $SPECS_DIR/experiment.yaml \
                gen_trt_engine.onnx_file=$RESULTS_DIR/export/changenet.onnx \
                gen_trt_engine.trt_engine=$RESULTS_DIR/gen_trt_engine/changenet.trt

In [ ]:
!tao deploy visual_changenet inference \
            -e $SPECS_DIR/experiment.yaml \
                inference.trt_engine=$RESULTS_DIR/gen_trt_engine/changenet.trt \
                inference.results_dir=$RESULTS_DIR/trt_inference

In [ ]:
!tao deploy visual_changenet evaluate \
            -e $SPECS_DIR/experiment.yaml \
                evaluate.trt_engine=$RESULTS_DIR/gen_trt_engine/changenet.trt \
                evaluate.results_dir=$RESULTS_DIR/trt_evaluate

In [ ]:
# Visualizing the sample images.
IMAGE_DIR = os.path.join(os.environ['HOST_RESULTS_DIR'], "trt_inference")
COLS = 2 # number of columns in the visualizer grid.
IMAGES = 4 # number of images to visualize.

visualize_images(IMAGE_DIR, num_cols=COLS, num_images=IMAGES)

This notebook has come to an end.